# Лабораторная работа 5
1. Использовать предобработанные тексты выбранного корпуса
2. Извлечь вектора BoW, TF-IDF, Word2Vec, BERT*
3. Сохранить предобученные модели
4. *Сохранить полученные вектора отдельным файлом (.py/ .csv)
5. *Предсказать сентимент отзывов используя тональные словари

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gensim > None

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# загрузка выборок
train_data = pd.read_csv('/content/drive/My Drive/data_train.csv', keep_default_na=False)
test_data = pd.read_csv('/content/drive/My Drive/data_test.csv', keep_default_na=False)
val_data = pd.read_csv('/content/drive/My Drive/data_val.csv', keep_default_na=False)

In [ ]:
# проверка пропущенных значений
train_data['processed_text'].isna().sum()

0

In [ ]:
train_data

,id_post,date,text,views,reactions,comments,engagement,target,processed_text
0,15223,2024-03-14 11:59:58+00:00,Продолжаем наш совместный с @MosGuru розыгрыш!...,226362,1372,85,0.006437,False,продолжать наш совместный mosguru розыгрыш он ...
1,17189,2024-06-11 09:50:39+00:00,"Клавиатуры добавили только тех стран, в которы...",167889,613,57,0.003991,False,клавиатура добавить тот страна который продава...
2,7599,2022-12-30 12:15:33+00:00,**В чипе A17 для iPhone 15 Apple сфокусируется...,182941,2289,0,0.012512,False,чип a iphone apple сфокусироваться энергоэффек...
3,4092,2021-07-01 08:52:18+00:00,Вышла публичная бета iOS 15. Ставить на основн...,20043,9,0,0.000449,False,выйти публичный бета ios ставить основный устр...
4,6762,2022-09-17 16:51:50+00:00,Вот так делаешь. И eSIM улетела на новый iPhone.,401874,5783,0,0.014390,False,делать esim улететь новый iphone
...,...,...,...,...,...,...,...,...,...
11203,17583,2024-07-02 16:20:42+00:00,**В Техасе есть точная копия стамбульской трас...,191088,1254,79,0.006976,False,техас точный копия стамбульский трасса формула...
11204,24380,2025-04-18 17:01:37+00:00,**Google добавит красок в чистый Android 16**\...,82788,477,78,0.006704,False,google добавить краска чистый android новый бе...
11205,8339,2023-02-28 11:03:01+00:00,**Redmi **[**показал**](https://weibo.com/3021...,165714,1507,0,0.009094,False,redmi зарядка вт нуль пять минута xiaomi разра...
11206,5016,2021-12-10 09:03:46+00:00,Мода на кроссовки в стиле тапочек набирает обо...,16392,21,0,0.001281,False,мода кроссовок стиль тапочки набирать оборот n...


## BoW

In [ ]:
import joblib
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# создание векторизатора
vectorizer_bow = CountVectorizer()
# обучение векторизатора
X_bow = vectorizer_bow.fit_transform(train_data['processed_text'])

In [ ]:
# размер
X_bow.shape

(11208, 30761)

In [ ]:
# преобразование валидационной и тестовой выборок
X_val_bow = vectorizer_bow.transform(val_data['processed_text'])
X_test_bow = vectorizer_bow.transform(test_data['processed_text'])

In [ ]:
vectorizer_bow.transform(['привет', 'пока']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# сохранение векторизатора
joblib.dump(vectorizer_bow, 'vectorizer_bow.pkl')

['vectorizer_bow.pkl']

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# создание векторизатора
vectorizer_tfidf = TfidfVectorizer()
# обучение векторизатора
X_tfidf = vectorizer_tfidf.fit_transform(train_data['processed_text'])

In [ ]:
X_tfidf.shape

(11208, 30761)

In [ ]:
# преобразование валидационной и тестовой выборок
X_val_tfidf = vectorizer_tfidf.transform(val_data['processed_text'])
X_test_tfidf = vectorizer_tfidf.transform(test_data['processed_text'])

In [ ]:
joblib.dump(vectorizer_tfidf, 'vectorizer_tfgif.pkl')

['vectorizer_tfgif.pkl']

## Word2Vec

In [ ]:
import gensim.downloader as api
from gensim.models import Word2Vec

In [ ]:
corpus = [text.split() for text in train_data['processed_text']]
# обучение модели
w2v_model = Word2Vec(sentences=corpus, vector_size=300, window=5, min_count=1, epochs=300)

In [ ]:
# размер словаря
len(w2v_model.wv.key_to_index)

30809

In [ ]:
# сколько раз слово встретилось в корпусе
w2v_model.wv.get_vecattr('apple', 'count')

4117

In [ ]:
print(w2v_model.wv['tesla'])

[-2.80457282e+00 -2.39453658e-01  9.60213840e-01 -2.61801863e+00
  2.80431843e+00  7.16488779e-01 -3.33885789e+00 -1.52542937e+00
 -6.15169287e-01  8.33488524e-01 -1.80493855e+00 -2.12677643e-01
 -7.39630878e-01  2.12120008e+00  2.08864480e-01  1.58285952e+00
  1.55880481e-01  4.58316517e+00  3.15609813e+00  5.39344788e-01
  1.33698261e+00  1.95126459e-01 -3.42863113e-01  2.70251483e-01
 -2.28261173e-01  2.30955386e+00  2.90446138e+00 -1.56988788e+00
  2.05438399e+00 -1.68829536e+00  1.99555993e+00 -2.65447021e-01
 -4.32673931e-01 -2.46114707e+00  1.79986253e-01 -1.73464131e+00
  3.30909419e+00 -7.67053306e-01  2.54152827e-02 -2.28594470e+00
 -1.17286408e+00  1.85270894e+00 -2.86348391e+00  1.16738260e+00
 -7.93689430e-01  1.95539519e-01  6.32003450e+00  3.22554040e+00
  3.25535369e+00 -6.12738371e-01  1.86115575e+00 -7.33445585e-02
 -3.55968714e-01  2.27732348e+00 -3.49825549e+00 -1.33349967e+00
  8.28257203e-01  1.82215643e+00 -2.60080481e+00 -3.44841421e-01
 -1.38487780e+00  1.03527

In [ ]:
# ближайшие слова
w2v_model.wv.most_similar('apple')

[('компания', 0.6862130761146545),
 ('год', 0.6665082573890686),
 ('который', 0.6462792158126831),
 ('новый', 0.642440140247345),
 ('iphone', 0.5894425511360168),
 ('pro', 0.5786615014076233),
 ('устройство', 0.5593161582946777),
 ('смартфон', 0.5080121159553528),
 ('ещё', 0.4936359226703644),
 ('айфон', 0.46228641271591187)]

In [ ]:
# ближайшие слова
w2v_model.wv.most_similar('iphone')

[('pro', 0.6978105306625366),
 ('айфон', 0.6573114991188049),
 ('apple', 0.5894425511360168),
 ('смартфон', 0.5610325932502747),
 ('новый', 0.5555613040924072),
 ('год', 0.5187150835990906),
 ('устройство', 0.49643054604530334),
 ('версия', 0.48064249753952026),
 ('который', 0.47567084431648254),
 ('max', 0.46194207668304443)]

In [ ]:
# ближайшие слова
w2v_model.wv.most_similar('смартфон')

[('устройство', 0.6072728037834167),
 ('iphone', 0.5610325932502747),
 ('apple', 0.5080121755599976),
 ('айфон', 0.49448591470718384),
 ('который', 0.4904525578022003),
 ('pro', 0.4854702651500702),
 ('год', 0.4796941578388214),
 ('новый', 0.46409982442855835),
 ('ещё', 0.4067133069038391),
 ('компания', 0.3912524878978729)]

In [ ]:
# косинусная близость
w2v_model.wv.similarity('apple', 'iphone')

0.5894425

In [ ]:
w2v_model.wv.similarity('айфон', 'смартфон')

0.49448594

In [ ]:
# функция для векторизации
def vectorize(text, model):
    vectors = []
    for t in text:
        # если слово есть в словаре w2v - берем его вектор
        # если слова нет - вставляем нудевой вектор
        vectors_word = [model.wv[word] if word in model.wv else np.zeros(300) for word in t.split()]
        # усреднение
        if vectors_word:
            mean_vector = np.mean(vectors_word, axis=0)
        else:
            mean_vector = np.zeros(300)
        vectors.append(mean_vector)

    return np.array(vectors)

In [ ]:
#  векторизация обучающей, валидационной и тестовой выборок
X_train_w2v = vectorize(train_data['processed_text'], w2v_model)
X_val_w2v   = vectorize(val_data['processed_text'], w2v_model)
X_test_w2v  = vectorize(test_data['processed_text'], w2v_model)

In [ ]:
# проверка размера векторов
print(X_train_w2v.shape)

(11208, 300)


In [ ]:
w2v_model.save('/content/drive/My Drive/word2vec.model')

In [ ]:
# сохраняем вектора
pd.DataFrame(X_train_w2v).to_csv('/content/drive/My Drive/X_train_w2v.csv', index=False)
pd.DataFrame(X_val_w2v).to_csv('/content/drive/My Drive/X_val_w2v.csv', index=False)
pd.DataFrame(X_test_w2v).to_csv('/content/drive/My Drive/X_test_w2v.csv', index=False)